In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from datasets import Dataset 
import pandas as pd
import numpy as np
import torch
import evaluate 
import os 

In [2]:
data = pd.read_csv('../data/data_preprocessed.csv')
dataset = Dataset.from_pandas(data)

In [3]:
# Loading pre-trained model and tokenizer 

model = AutoModelForSequenceClassification.from_pretrained('JamesH/Movie_review_sentiment_analysis_model')
tokenizer = AutoTokenizer.from_pretrained('JamesH/Movie_review_sentiment_analysis_model')
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [4]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

In [5]:
# Tokenizing the dataset before train/test splitting and selecting a small subset for model fine tuning 

tokenized_ds = dataset.map(tokenize_function, batched=True)
tokenized_ds = tokenized_ds.train_test_split(test_size=0.2)

small_train = tokenized_ds['train'].shuffle(seed=13).select(range(200))
small_test = tokenized_ds['test'].shuffle(seed=13).select(range(200))

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
current_dir = os.getcwd()
main_dir = os.path.abspath(os.path.join(current_dir, '..'))
models_dir = os.path.abspath(os.path.join(main_dir, 'models')) # Creating a new directory for saving models 

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred 
    predictions = np.argmax(logits, axis=-1)
    
    # Evaluation metrics for classification
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average=None)
    acc = accuracy_score(labels, predictions)
    auc = roc_auc_score(labels, logits[:, 1])

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "auc": auc
        }

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=models_dir,
    save_strategy='no',
    eval_strategy='epoch',
    num_train_epochs=1,
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\sherr\anaconda3\envs\lhl_env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Auc
1,No log,0.263901,0.940000,[0.93617021 0.94339623],[0.95652174 0.92592593],[0.91666667 0.96153846],0.985777


TrainOutput(global_step=100, training_loss=0.4308546829223633, metrics={'train_runtime': 865.8761, 'train_samples_per_second': 0.231, 'train_steps_per_second': 0.115, 'total_flos': 43675162884048.0, 'train_loss': 0.4308546829223633, 'epoch': 1.0})

In [10]:
trainer.evaluate()

C:\Users\sherr\anaconda3\envs\lhl_env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.2639005184173584,
 'eval_accuracy': 0.94,
 'eval_f1': array([0.93617021, 0.94339623]),
 'eval_precision': array([0.95652174, 0.92592593]),
 'eval_recall': array([0.91666667, 0.96153846]),
 'eval_auc': 0.9857772435897436,
 'eval_runtime': 194.6869,
 'eval_samples_per_second': 1.027,
 'eval_steps_per_second': 0.514,
 'epoch': 1.0}

In [14]:
def op_compute_metrics(eval_pred):
    logits, labels = eval_pred 
    predictions = np.argmax(logits, axis=-1)
    
    # Evaluation metrics for classification
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average=None)
    acc = accuracy_score(labels, predictions)

    return {
        "accuracy": float(acc),
        "precision_class_0": float(precision[0]),
        "precision_class_1": float(precision[1]),
        "recall_class_0": float(recall[0]),
        "recall_class_1": float(recall[1]),
        "f1_class_0": float(f1[0]),
        "f1_class_1": float(f1[1])
        }

In [15]:
# Optimizing hyperparameters 

def model_init():
    return AutoModelForSequenceClassification.from_pretrained('JamesH/Movie_review_sentiment_analysis_model')

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_test,
    data_collator=data_collator,
    compute_metrics=op_compute_metrics,
)
# Define hyperparameter search space
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "warmup_steps": trial.suggest_int("warmup_steps", 0, 100),
    }


best_trial = trainer.hyperparameter_search(
    direction="maximize",  # Maximize metric 
    hp_space=hp_space,
    backend="optuna",      # Use Optuna as the backend
    n_trials=2            # Number of trials to run
)

print(f"Best trial: {best_trial}")

[I 2025-07-03 14:38:58,153] A new study created in memory with name: no-name-be22af6a-bfd2-4003-85f8-dc22b59180fa
C:\Users\sherr\anaconda3\envs\lhl_env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision Class 0,Precision Class 1,Recall Class 0,Recall Class 1,F1 Class 0,F1 Class 1
1,No log,0.527501,0.895000,1.000000,0.832000,0.781250,1.000000,0.877193,0.908297


[I 2025-07-03 14:54:17,835] Trial 0 finished with value: 6.293739925687582 and parameters: {'learning_rate': 3.268307323769655e-05, 'warmup_steps': 65}. Best is trial 0 with value: 6.293739925687582.
C:\Users\sherr\anaconda3\envs\lhl_env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision Class 0,Precision Class 1,Recall Class 0,Recall Class 1,F1 Class 0,F1 Class 1
1,No log,0.497807,0.905000,1.000000,0.845528,0.802083,1.000000,0.890173,0.916300


[I 2025-07-03 15:09:21,424] Trial 1 finished with value: 6.359084758493876 and parameters: {'learning_rate': 3.2306249035382145e-05, 'warmup_steps': 63}. Best is trial 1 with value: 6.359084758493876.


Best trial: BestRun(run_id='1', objective=6.359084758493876, hyperparameters={'learning_rate': 3.2306249035382145e-05, 'warmup_steps': 63}, run_summary=None)


In [23]:
# Creating a subdirectory for the tuned model and saving it 

ft_model = os.path.abspath(os.path.join(models_dir, 'finetuned_model'))
os.makedirs(ft_model, exist_ok=True)

model.save_pretrained(ft_model)
tokenizer.save_pretrained(ft_model)         